# Verificação Indutiva

Na aula passada vimos como modelar sistemas dinâmicos com *First-order Transition Systems* (FOTSs) e como verificar propriedades de segurança e animação sobre estes sistemas com *Bounded Model Checking* (BMC). O procedimento de BMC verifica se uma propriedade é válida num FOTS considerando apenas um número limitado de estados de execução. Se quisermos verificar uma propriedade para qualquer execução não limitada do FOTS temos que usar um procedimento alternativo. Nesta aula vamos usar indução para fazer verificação em traços não limitadados.

## Verificação indutiva de invariantes

No caso da verificação de propriedades de segurança $G\ \phi$, para verificar o invariante $\phi$ por indução temos que verificar as seguintes condições:
- $\phi$ é válido nos estados iniciais, ou seja, $\mathit{init}(s) \rightarrow \phi(s)$
- Para qualquer estado, assumindo que $\phi$ é verdade, se executarmos uma transição, $\phi$ continua a ser verdade no próximo estado, ou seja, $\phi(s) \wedge \mathit{trans}(s,s') \rightarrow \phi(s')$.

### Exercício 1

Implemente a função `induction_always` para verificação de invariantes por indução. 
A função recebe como argumento uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e o invariante.

Note que terá que testar a validade das duas condições acima recorrendo à satisfiabilidade, ou seja, usando o solver para encontrar contra-exemplos, devendo o procedimento reportar qual das propriedades falha. Por exemplo, no caso da primeira deve procurar uma valoração que satisfaça $\mathit{init}(s) \wedge \neg \phi(s)$.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT


def induction_always(declare,init,trans,inv):
    with Solver(name="z3") as s:
    # declarar 2 cópias de estado -> s e s'
    
        state0 = declare(0) # seja s = 0
        state1 = declare(1) # e    s = 1
    
        # testar o estado inicial: init(𝑠)→𝜙(𝑠)
    
        s.push()
        #acrescentamos a condição init(𝑠)∧¬𝜙(𝑠).
        s.add_assertion(init(state0))
        s.add_assertion (Not(inv(state0)))
    
        # se for satisfazivel então a propriedade é falsa
        if s.solve():
            print("A propriedade não é válida no estado inicial.")
            for v in state0:
                print(v,"=",s.get_value(state0[v]) )
            return   # só return e terminamos pois falhou a propriedade falta apenas s.pop()
        s.pop()
    
        # testar o passo indutivo: 𝜙(𝑠)∧trans(𝑠,𝑠′)→𝜙(𝑠′).
        s.push()
        s.add_assertion(And(inv(state0), trans(state0,state1)))
        s.add_assertion (Not(inv(state1)))
        
        if s.solve():
            print("O passo indutivo não preserva a propriedade.")
            for v in state0:
                print(v,"=",s.get_value(state0[v]) )
            return            # só return e terminamos pois falhou a propriedade falta apenas s.pop()
        # caso ambos no estado init e no passo indutivo sejam válidos então não vamos retornar nada
        print("A propriedade é sempre válida.")
        s.pop()
    
    

Usando este procedimento podemos tentar verificar a propriedade $G (x \ge 0)$ sobre o programa

```Python
{ x >= 3 }
0: while (x>0):
1:    x = x-1
2: stop
```

In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),INT)
    return state

def init(state):
    return And(Equals(state['pc'],Int(0)), state['x']>= Int(3))

def trans(curr,prox):
    t01 = And(Equals(curr['pc'],Int(0)),curr['x'] > Int(0), Equals(prox['pc'],Int(1)), Equals(prox['x'],curr['x']))
    t02 = And(Equals(curr['pc'],Int(0)),curr['x'] <= Int(0), Equals(prox['pc'],Int(2)), Equals(prox['x'],curr['x']))
    t10 = And(Equals(curr['pc'],Int(1)),Equals(prox['pc'],Int(0)),Equals(prox['x'],curr['x']-Int(1)))
    t22 = And(Equals(curr['pc'],Int(2)),Equals(prox['pc'],Int(2)),Equals(prox['x'],curr['x']))
    return Or(t02,t01,t10,t22)

def nonnegative(state):
    return (state['x'] >= Int(0))

induction_always(declare,init,trans,nonnegative)

O passo indutivo não preserva a propriedade.
pc = 1
x = 0


Como se pode verificar, a propriedade anterior, embora seja verdade, não pode ser verificada por indução, pois o passo indutivo falha quando $\mathit{pc} = 1$ e $x = 0$. De facto, este contra-exemplo para o passo de indução não é realista, dado que este estado não é acessível a partir do estado inicial. Para verificarmos esta propriedade por indução temos que encontrar um invariante $\phi$ que seja mais forte que o desejado, ou seja, $\phi(x,\mathit{pc}) \rightarrow x \ge 0$, e que seja verificável por indução.

### Exercício 2

Encontre uma propriedade mais forte que $x \ge 0$ e que seja verificável por indução. Sugestão: pode ser útil usar a funcção `Ite` do pySMT, que corresponde a um if-then-else lógico.

In [3]:
def nonnegativeStronger(state):
    return (Ite(Equals(state['pc'],Int(1)),state['x']>Int(0), state['x']>=Int(0)))

induction_always(declare,init,trans,nonnegativeStronger)

A propriedade é sempre válida.


### Exercício 3

Utilize o SMT solver para verificar que essa propriedade é realmente mais forte que $x \ge 0$.

In [4]:
# Usando o is_valid por exemplo
state = declare(10)  # estado qualquer nao tem de ser o inicial dai se chamar só state e ser o estado 10
res = is_valid(Implies(nonnegativeStronger(state),nonnegative(state))) 
print(res)

# De uma outra forma e equivalente
if is_sat(Not(Implies(nonnegativeStronger(state),nonnegative(state)))):
    print("Não.")
else:
    print("Sim.")

True
Sim.


Nem sempre é fácil encontrar um invariante indutivo mais forte que o desejado. Um procedimento de verificação alternativo que pode ser utilizado nesses casos é a designada $k$-indução. A ideia é generalizar a indução simples assumindo no passo indutivo que o invariante é válido nos $k$ estados anteriores. Claro que neste caso também será necessário verificar que a propriedade é válida nos $k$ primeiros estados. Com esta generalização é possível verificar que $x \ge 0$ é um invariante recorrendo a $k$-indução com $k=2$.

### Exercício 4

Implemente o método `kinduction_always` para verificação de invariantes por $k$-indução.

In [59]:
def kinduction_always(declare,init,trans,inv,k):
    with Solver(name="z3") as s:
        
        # declarar k+1 cópias de estado -> s e s'
    
        traco = [declare(i) for i in range(k+1)]
        
        # testar os  k estados iniciais: init(𝑠)→𝜙(𝑠)
        s.push()
        s.add_assertion(init(traco[0]))
        for i in range(k-1):
            s.add_assertion(trans(traco[i],traco[i+1]))
        s.add_assertion(Or([Not(inv(traco[i])) for i in range(k)]))
        
        # se for satisfazivel então a propriedade é falsa
        if s.solve():
            print("A propriedade não é válida nos k estados iniciais.")
            for v in traco[0]:
                print(v,"=",s.get_value(traco[0][v]) )
            return   # só return e terminamos pois falhou a propriedade falta apenas s.pop()
        s.pop()

        # testar o passo k-indutivo
        s.push()
        #!! vamos negar o que temos na folha ver aula 10/11
        for i in range(k):
            s.add_assertion(inv(traco[i]))
            s.add_assertion(trans(traco[i],traco[i+1]))
        s.add_assertion(Not(inv(traco[k])))        
        
        if s.solve():
            print("O passo %s indutivo não preserva a propriedade." % k)
            print("O ponto onde falha começa aqui:")
            for v in traco[0]:
                print(v,"=",s.get_value(traco[0][v]) )
            return   # só return e terminamos pois falhou a propriedade falta apenas s.pop()
        s.pop()
        
        print("A propriedade é sempre válida.  (k = %d)" % k)
        
kinduction_always(declare,init,trans,nonnegative,2)  # com 1 tinhamos o que acontecia no ex.1

#corrigir este kinduction, deveria dar erro no exercicio 5 para k=2 e não dá
#erro estava em  s.add_assertion(Not(inv(traco[k]))), eu tinha  s.add_assertion(Not(inv(traco[i])))  :/

A propriedade é sempre válida.  (k = 2)


## Verificação de propriedades de animação por indução

Como vimos, podemos verificar propriedades de animação do tipo $F\ \phi$ usando BMC. Mais uma vez, se quisermos verificar estas propriedades para qualquer execução ilimitada temos que usar um procedimento alternativo. Uma possibilidade consiste em reduzir a verificação dessas propriedades à verificação de uma propriedade de segurança, mais concretamente um invariante, que possa ser verificado por indução.

Por exemplo, uma técnica que pode ser usada para verificar uma propriedade de animação do tipo $F\ (G\ \phi)$, passa por descobrir um *variante* $V$ que satisfaz as seguintes condições:
- O variante nunca é negativo, ou seja, $G\ (V(s) \ge 0)$
- O variante descresce sempre (estritamente) ou atinge o valor 0, ou seja, $G\ (\forall s' . \mathit{trans}(s,s') \rightarrow (V(s') < V(s) \vee V(s') = 0))$
- Quando o variante é 0 verifica-se necessariamente $\phi$, ou seja, $G\ (V(s)=0 \rightarrow \phi(s))$

A terminação de um programa é um dos exemplos de propriedade que encaixa neste padrão. Por exemplo, para o programa acima a terminação pode ser expressa por $F\ (G\ \mathit{pc}=2)$ (note que devido ao lacete no estado final, o programa permanece indefinidamente no estado final).

Note que a segunda condição recorre a uma quantificação universal sobre os possíveis estados alcançáveis por uma transição a partir de cada estado.

### Exercício 5

Descubra o variante que permite provar por indução que o programa acima termina. Encontre também os menores valores de $k$ que permitem provar as condições acima por $k$-indução.

In [60]:
def variante(state):
    return Int(2)*state['x'] -state['pc'] + Int(2)

#Agora falta provar para k valores como é pedido

# 𝐺 (𝑉(𝑠)≥0)
def nonnegativeV(state):
    return variante(state)>=Int(0)

# 𝐺 (∀𝑠′.trans(𝑠,𝑠′)→(𝑉(𝑠′)<𝑉(𝑠)∨𝑉(𝑠′)=0))
kinduction_always(declare,init,trans,nonnegativeV,3) # só deveria aceitar de 3 para cima, e já aceita após corrigir erro

def decrescente(state):
    newstate = declare(-1)
    return ForAll(list(newstate.values()),
                  Implies(trans(state,newstate),Or(variante(newstate)<variante(state),
                                                   Equals(variante(newstate),Int(0))  )  )  )
kinduction_always(declare,init,trans,decrescente,3)

# 𝐺 (𝑉(𝑠)=0→𝜙(𝑠))  queremos pc= 2 quando variante for 0
def util(state):
    return Implies(Equals(variante(state),Int(0)),Equals(state['pc'],Int(2))) 

kinduction_always(declare,init,trans,util,3) # 1 tem de falhar e agora falha

A propriedade é sempre válida.  (k = 3)
A propriedade é sempre válida.  (k = 3)
A propriedade é sempre válida.  (k = 3)


### Exercício 6

Encontrar um variante que decresça em todas as transições nem sempre é fácil. Podemos relaxar a segunda condição acima e exigir que o variante apenas tenha que decrescer estritamente a cada $l$ transições. Neste caso dizemos que temos um *lookahead* de $l$. Altere a definição da função decrescente por forma a considerar um lookahead de 2 e encontre um variante mais simples que o anterior que lhe permita verificar a terminação do programa.

In [68]:
def variante2(state):
    return state['x']-state['pc'] + Int(2)

def nonnegativeV(state):
    return variante(state)>=Int(0)

kinduction_always(declare,init,trans,nonnegativeV,3)

def util(state):
    return Implies(Equals(variante(state),Int(0)),Equals(state['pc'],Int(2))) 

kinduction_always(declare,init,trans,util,2) # util deve tar mal nao faço ideia

#ta correta !!!!
def decrescente2(state):
    newstate1 = declare(-1)
    newstate2 = declare(-2)
    return ForAll(list(newstate1.values())+list(newstate2.values()),
                 Implies(And(trans(state,newstate1), trans(newstate1,newstate2) ),Or(variante(newstate2)<variante(state),
                 Equals(variante(newstate2),Int(0))) ))

kinduction_always(declare,init,trans,decrescente2,3) # deveria ser aceite corrigir esta 6

# completar decrescente2 acima, acho que já esta correta mas é melhor verificar

#algo mais radical

def variante(state):
    return Ite(Equals(state['pc'],Int(2)), Int(0),state['x']-state['pc'])

kinduction_always(declare,init,trans,decrescente2,1)

A propriedade é sempre válida.  (k = 3)
O passo 5 indutivo não preserva a propriedade.
O ponto onde falha começa aqui:
pc = 0
x = 3
A propriedade é sempre válida.  (k = 3)
A propriedade é sempre válida.  (k = 1)
